# Introduction

Today's data set involves analysis of electroencephalography (EEG) data measured in human participants. Typcially, the set-up for such an experiment looks something like this:
![caption](Human-Electrophys.jpg)
Each wire is connected to an individual electrode which collects the voltage at that location on the scalp in comparison to a reference voltage.  Changes in voltage over time result in a waveform with peaks and troughs, the timing (a.k.a. latency) and amplitude of which can provide important clues as to how different brain areas are responding to auditory input (or visual, somatosensory, or cognitive input for that matter!).  A set-up such as the one shown above can be used to assess a wide range of neurophysiological processes from simple neural detection of stimuli all the way up to studying the complex interactions between cognition and sensory function. In far-field electrophysiology set-ups such as this, we can isolate the neural response to the stimulus from the unrelated neural activity by presenting many trials of the same stimulus, and then averaging these responses together.  Then, by averaging together responses to these trials (called epochs), the components of the response that are timelocked to the stimulus are emphasized while the background noise that is not time-locked to the stimulus averages out and falls away. Thus, the data that we will examine today falls into a special class of EEG known as "event-related potentials (ERP)" or "evoked potentials (EP)", as opposed to on-going EEG measures such as those that may be recorded for purposes such as sleep studies.
 
The current data set targets an ERP response known as the P300. P300s are obtained using what is known as an "oddball" paradigm which utilizes two stimuli that differ in some feature (e.g. frequency, intensity, etc.).  One of the stimuli is presented in the majority of trials (usually around 80%) while the other stimulus is randomly presented in the remaining trials.  This infrequently presented stimulus is the "oddball" (hence the name "oddball" paradigm), and the brain responds to it by producing a large positive voltage occuring approximately 300 ms following the onset of the stimulus.  Responses typcially look something like this:
![caption](P300-Figure.jpg)

The P300 stimulus contrast used in today's data set was a frequency change, with the standard tone being 500 Hz and the oddball tone being 1000 Hz.  Participants completed two different tasks or conditions.  In one condition (the "Go" condition), participants were asked to press a button whenever they heard the oddball stimulus.  In the other condition (the "NoGo" condition), participants were asked to press a button every time they heard the standard stimulus and withold pressing the button when they heard the oddball stimulus. Thus, the stimulus contrast was the same for both conditions, but the cognitive effort associated with the task differed, with the NoGo condition being more cognitively demanding than the Go condition.  Comparison of P300 responses between these two conditions will give us information about the neural encoding of an unexpected change in pitch and how increasing cognitive load impacts this process.   

To simplify the dataset, we will only work with data collected from the electrode location "Pz" which is positioned centrally over the parietal cortex.  P300 responses tend to be largest when measured at this location.  The goal of today's exercises is as follows:

1. start with all of the responses to each oddball trial for each participant for each condition

2. average the trials together according to subject and condition, being sure to omit trials with noisy data that failed artifact rejection testing,

3. select and plot average responses from individual participants as well as data averaged across all participants for each condition

To do this, we will use python packages inlcuding pandas, numpy and seaborn. Note how we import them below. We will use `pd`, `sns`, and `np` as shorthand for these packages.

Go ahead and import these packages and set up your display options (the number of rows and columns that you'll see when you look at the dataframe) using the code below.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

pd.options.display.max_rows = 7
pd.options.display.max_columns = 7

# Loading and Organizing the Data

The data that we will use is stored in a file called *EP_data_V2.csv*. Run the cell below to create a variable, `filename` with the path to the file.

In [ ]:
filename = 'EP_data_V2.csv'

## Exercise 1

If you were to open the file *EP_data_V2.csv*, you would see that the first line begins with the four words 'subject', 'condition', 'trial', and 'accept' (if you want to view the contents of the file, you can go ahead and double-click on it and Jupyter will open it in a new tab).  The data in the spreadsheet are arranged such that the first row contains the headings listed above in the first four cells followed by numbered samples starting at 0 and ending at 1300.  These numbered samples correspond to the 1300 samples in time beginning 200 ms prior to stimulus onset and ending 1100 ms post stimulus onset measured using a sampling rate of 1000 Hz.  Subsequent rows contains the values assocated with the headers followed by numbers indicating the voltages measured across all 1300 samples across time. 

Our first task is to read in csv file making sure to use the data in the 'subject', 'condition', 'trial', and 'accept' columns using the pd.read_csv function that we learned last week.  Last week, we only dealt with two sets of tabular data. The first had a single column which we used as an index. The second we just allowed Pandas to automatically assign an index. In the data we're working on this week, it makes more sense to combine these four columns to create an index that uniquely identifies each row.  How do we do it?  To get a hint, use the help function that we learned about last week, paying special attention to the *index_col* function.

In [ ]:
pd.read_csv?

So, it looks like we can specify multiple columns by passing a list as an argument to the `index_col` parameter. Last week we only specified a single column for the exposure data table. Using this information, go ahead and read the *EP_data_V2.csv* file into a new dataframe. Make sure to call the new dataframe **EP_data**.

*Hint!  the index_col function can accept either column names or column numbers (starting at 0).*

In [ ]:
# Answer
EP_data = pd.read_csv(filename, index_col=['subject', 'condition', 'trial', 'accept'])
EP_data

The documentation for the `read_csv` function indicates that when a list of columns is provided via `index_col` , a dataframe with a MultiIndex is returned.  In the present example, our MultiIndex contains four different *levels* consisting of the four column names that we passed to `read_csv`.  Having a MultiIndex with multiple levels (as opposed to the single simple index we used last week) will make it easier for us to pull out data from the dataframe across multiple levels, for example, pulling out the 13th trial from subject 019 in the NoGo condition.  This will come in handy in our upcoming examples!

Overall, our new data frame looks pretty good, with all rows of waveform data nicely aligned with associated subject, condition, trial, and accept values.  But I'll give you a hint about this data: the lowest subject number is 6, and the subject number 43 is actually the highest number!  We can organize the data more conviently by using the pandas dataframe function `sort_index` to sort the data in our indices in a more logical way.  Depending on what you're doing with the data, it's not mandatory to sort the index, but some operations, such as extracting data using `loc`, require a sorted index. 

Run the following code and compare with the previous output of EP_data to see how using the `sort_index` function changed the dataframe.

In [ ]:
EP_data.sort_index(inplace=True)
EP_data

In case you're wondering what the meaning of "inplace = True" is, this means we are telling pandas to keep the reorganized data in our current data frame as opposed to making a new dataframe with the sorted values.

Lastly, there's a common quirk of electrophysiological data that responses are often obtained with positive values going downwards and negative values going upwards.  Hence, to make our plotting and analysis easier, let's go ahead and flip the sign of the data in our new data frame using the code below:

In [ ]:
EP_data = -EP_data
EP_data

# Extracting Specific Data from the Dataframe

Now that we've got the dataframe imported and organized in a logical way, we are ready to start extracting some data.

## Exercise 2

Extract data from subject 19. Remember that the syntax for the 'loc' indexer is dataframe.loc[rows,cols]

In [ ]:
# Answer
data = EP_data.loc[19, :]
data

## Exercise 3

You can extract multiple subjects using what's known as slice notation. Recall that the slice notation syntax is `[start:stop]`. When using dataframe `loc`:

    dataframe.loc[start_row:stop_row, start_column:stop_column]

You can also combine the `[start:stop]` notation with the : operator so that you can to specify everything in a row, column, or index as we just did in Exercise 2.  For the next exercise, apply slice notation to extract all the data associated with subjects 6 through 19.

In [ ]:
# Answer
data = EP_data.loc[6:20, :]
data

We can also selectively extract data across multiple levels.  This can be done by specifying the levels of interest using a special object provided by Pandas called an *IndexSlice* and assigning the "slice" of data to a new variable name. Here, you're now using two levels of the index to specify that you want data from subject 19 in the NoGo condition. Is it a dataframe or series?

In [ ]:
row_slice = pd.IndexSlice[19, 'NoGo']
data = EP_data.loc[row_slice, :]
data

## Exercise 4 
Now, practice Slice notation by extracting data from subject 6 for the Go condition on trial 5 with an 'accept' value of True.  Is it a dataframe or a series?

In [ ]:
# Answer
row_slice = pd.IndexSlice[6, 'Go', 5, True]
data = EP_data.loc[row_slice, :] 
data 

# Changing Samples into a Meaningful Time Series

Let's take a look at this data.  Remember the dataframe method `plot` that we used last week? Let's use that to plot the accepted EEG response from subject 6 for the 'Go' condition on trial number 5.

In [ ]:
data.plot()

The x-axis isn't particularly meaningful here. It goes from sample 0 to sample 1300. We want to update it to show time in milliseconds (ms) from the stimulus onset, ranging from -200 to 1100 ms.  Recall that -200 indicates that the recording started 200 ms prior to the onset of the stimulus and continued until 1100 ms post-stimulus onset.  Data was sampled at a rate of 1000 samples per second from the continuous EEG signal.  

Next, using this information, we'll go through a series of exercises to change this x-axis from number of samples to time in milliseconds.

The number of columns is the number of samples that make up the waveform. So, let's begin by first counting them. We can get the size of the dataframe via the *shape* attribute.

In [ ]:
EP_data.shape

This is a two-element tuple. The first value indicates that we have 977 rows in our dataframe (representing each trial for each participant in each condition) and the second number indicates the number of columns in our dataframe (representing each sample from 0 to 1300).

## Exercise 5
In order to create an accurate time series based upon our samples, we need to extract the second value that represents the number of columns in our dataframe. Extract this value and assign it to a new variabile `n_cols`. 

In [ ]:
# Answer
n_cols = EP_data.shape[1]
n_cols

Remember that the data was sampled at a rate of 1000 samples per second.  Run the code below to assign the variable `sample_rate` to the sampling rate of 1000.

In [ ]:
sample_rate = 1000

First, let's make a 1D numpy array that represents the sample number, and call this `samples`.

In [ ]:
samples = np.arange(n_cols)
samples

## Exercise 6
Now, we can convert these numbered samples to time (in seconds) by dividing the samples by the sampling rate. Call this variable `time`.

In [ ]:
# Answer
time = samples / sample_rate
time

The unit of time is currently in seconds, but it will be easier to look at if it were convereted to milliseconds.  Next, convert `time` from seconds to milliseconds (continue to call this variable `time`).  

In [ ]:
# Answer
time = time * 1000
time

Notice that our time series goes from 0-1300 ms.  But recall that the data collection begins at 200 ms before stimulus onset. We want our time series to run from -200-1100 ms.  How do we shift the `time` array to reflect this?

In [ ]:
# Answer
time = time - 200
time

Good, now that we have our time array, we can assign it to the column lables. Try looking for an *attribute* that might represent the column labels (remember to type `dataframe.` and then press tab to get a list of possible attributes to choose from).

In [ ]:
# Answer
EP_data.columns = time
EP_data

Notice that the columns are now labeled with our time points rather than samples.  

Use the code below to regenerate the previous plot. How does it look now?

In [ ]:
row_slice = pd.IndexSlice[6, 'Go', 5, True]
data = EP_data.loc[row_slice, :] 
data.plot()

# Creating the Average Response from the Epoched Data

## Exercise 7
Recall that we only want to use data that has passed the Artifact Rejection test as indicated by a value of `True` in the `accept` column.  Now, extract all trials across all subjects where the `accept` column is True. Save it to a variable called `EP_good_data`.

In [ ]:
# Answer
row_slice = pd.IndexSlice[:, :, :, True]
EP_good_data = EP_data.loc[row_slice, :]
EP_good_data

Next, average across all trials for each subject and condition combination and call this `EP_mean_data`.  This can be accomplished by using the `groupby` function that we learned about last week. 

In [ ]:
# Answer
EP_mean_data = EP_good_data.groupby(['subject', 'condition']).mean()
EP_mean_data

# Making a pretty plot of the data with the seaborn package

## Exercise 8

Seaborn is a Python data visualization library based upon matplotlib.  It provides a high-level interface for drawing attractive and informative statistical graphics.  More about Seaborn can be learned here: https://seaborn.pydata.org/. Seaborn can make pretty plots with relatively little work from you, but first you need to get your data in the correct format. The format for the plot function we're using from </font> Seaborn is to have your x values in one column, your y values in another column, and then have an optional group column.

Lets look again at the current format of `EP_mean_data`

In [ ]:
EP_mean_data

What are the different columns? What are the index columns?

For our plot, we want the x values to be the timepoints in msec, the y values to be the amplitude of the EEG response, and we'd like to group by condition. Right now, all the time points are separated out into 1301 columns. We need to reshape the dataframe to get the timepoints into a single column. To start, we need to take `subject` and `condition` out of the index and make them available as columns</font>. We can do this using the `reset_index` method.

In [ ]:
EP_data_reshape = EP_mean_data.reset_index()
EP_data_reshape

Notice that subject and condition are no longer in the index. There is now a numbered index.

Now we need to get all the timepoints into a single column. We can do this using the `melt` function. This function takes data that is expressed in separate columns and converts it to a single column. You need to give the melt function the dataframe and tell it which columns identify unique portions of the data. These columns will not be reformatted. All other columns will be reformatted unless you specify which variables you want to reformat as value variables (by adding the argument `value_vars=`). You also want to assign column names to the two new columns you'll be creating. The variable we're melting are the timepoints. We can call this new column `Time`. The amplitude measurements drawn from the data contained in the cells are the values. We can call this column `Amplitude`. The syntax looks like this: `pd.melt(dataframe, id_vars=[], value_name=, var_name=[])`. Notice that that `value_name` takes one argument because `melt` will generate a single value column, while `var_name` takes a list because you can have mutiple value variables.

In [ ]:
# Answer
EP_data_reshape = pd.melt(EP_data_reshape, id_vars=['subject', 'condition'], value_name='Amplitude', var_name=['Time'])
EP_data_reshape

Now we are ready to plot Time versus Amplitude in seaborn with the condition indicated by color. The syntax looks like this: `sns.lineplot(x= , y= , hue= , data= , ci=, err_style='band')` You need to fill in the column for the x values, the column for the y values, the column for the hue (color), the dataframe to use, and the confidence interval to use for the error bands (68%). The confidence interval is generated through bootstrapping. This consists of generating many alternative datasets from the experimental data by randomly drawing observations with replacement. The variability across these datasets is assumed to approximate sampling error and is used to compute bootstrap confidence intervals. Use ci=68 to get the standard error of the mean.

*This will take a while to run because it's estimating the 68% confidence interval for each individual timepoint. Don't worry about the pink-shaded FutureWarning.*

In [ ]:
# Answer
sns.lineplot(x='Time', y='Amplitude', hue='condition', data = EP_data_reshape , ci=68, err_style='band')

Let's make the plot a little more meaningful by customizing the axis labels and giving it a plot title. Note that `sns.lineplot` returns the axes containg the plot, so you can further fine-tune the appearance of the axes.

In [ ]:
ax = sns.lineplot(x='Time', y='Amplitude', hue='condition', data = EP_data_reshape , ci=68, err_style='band')
ax.set(xlabel='Time (msec)', ylabel=r'Amplitude (μV)',title = 'P300 Waveforms By Condition')


# Bonus Exercise 


We've talked a lot about extracting subsets of data based on rows.
Now, let's talk about also extracting data based on columns. How would you extract a region of interest (ROI) in terms of time?  

Recall that peak latency and amplitude measures are often used to assess ERP responses.  In the next two exercises, we'll examine how P300 amplitude differs across participants by extracting data in a specific time range corresponding to the P300 peak.  

## Exercise 9

Extract data for the 'Go' condition for each subject in the time range of 200 to 400 ms, where we observed the P300 peak in the figure that we generated above.  We will do this using the `EP_mean_data` dataframe.

In [ ]:
# Answer
row_slice = pd.IndexSlice[:, 'Go']
EP_data_ROI = EP_mean_data.loc[row_slice, 200:400] 

## Exercise 10

Now, make a barplot of the average response amplitude within that region for each subject in the 'Go' condition.  This will allow us to explore differences in response amplitude over the 200-400 ms range across subjects in this condition.


In [ ]:
# Answer
EP_data_ROI_mean = EP_data_ROI.mean(axis=1)
EP_data_ROI_mean.plot.bar()